In [5]:
import numpy as np
import datajoint as dj
import blob_transformation as bt
import pandas as pd
dj.blob.use_32bit_dims = True

In [6]:
# grab a session to play with
sess_id = 897749
animal_id = 'R502'
date = pd.Timestamp("2022-11-30")

In [20]:
# import peh & transform blob -> dict
session_key = {'sessid': sess_id}
bdata = dj.create_virtual_module("bdata", "bdata")
peh_blob = (bdata.ParsedEvents & session_key).fetch(as_dict=True)
peh_dict = bt.transform_blob(peh_blob[0]['peh'])

In [66]:
peh_dict[0] # events for a single trial

{'pokes': {'C': array([459.670004, 459.762304]),
  'L': array([457.195704, 458.241504]),
  'R': array([[       nan, 451.626404],
         [458.748004, 459.464804],
         [459.925704,        nan]]),
  'starting_state': {'C': 'out', 'L': 'out', 'R': 'in'},
  'ending_state': {'C': 'out', 'L': 'out', 'R': 'in'}},
 'waves': {'cp_fixation_wave': array([], shape=(0, 2), dtype=float64),
  'sa_sound_wave': array([459.770004, 460.070004]),
  'sb_sound_wave': array([460.170004, 460.470004]),
  'go_sound_wave': array([460.480004, 462.480004]),
  'starting_state': {'cp_fixation_wave': array([], shape=(0, 0), dtype=float64),
   'sa_sound_wave': 'out',
   'sb_sound_wave': 'out',
   'go_sound_wave': 'out'},
  'ending_state': {'cp_fixation_wave': array([], shape=(0, 0), dtype=float64),
   'sa_sound_wave': 'out',
   'sb_sound_wave': 'out',
   'go_sound_wave': 'out'}},
 'states': {'state_0': array([[       nan, 448.424204],
         [462.970704,        nan]]),
  'check_next_trial_ready': array([[     

In [33]:
# get some info on the peh dict (print in order of hierarchy)
itrial = 0
for key, value in peh_dict[itrial].items():
    print(f"1st level key: {key}")

for key, value in peh_dict[itrial].items():
    for key2, value2 in value.items():
        print(f"2nd level key: {key2}")

for key, value in peh_dict[itrial].items():
    for key2, value2 in value.items():
        try: 
            for key3, value3 in value2.items():
                print(f"3rd level key + val: {key3}, {value3}")
        except:
            None

1st level key: pokes
1st level key: waves
1st level key: states
2nd level key: C
2nd level key: L
2nd level key: R
2nd level key: starting_state
2nd level key: ending_state
2nd level key: cp_fixation_wave
2nd level key: sa_sound_wave
2nd level key: sb_sound_wave
2nd level key: go_sound_wave
2nd level key: starting_state
2nd level key: ending_state
2nd level key: state_0
2nd level key: check_next_trial_ready
2nd level key: wait_for_cpoke
2nd level key: cpoke
2nd level key: wait_for_sounds_to_end
2nd level key: early_spoke_state
2nd level key: give_reward
2nd level key: wait_for_spoke
2nd level key: hit_state
2nd level key: drink_state
2nd level key: temp_error_state
2nd level key: wait_for_spoke_retry
2nd level key: retry_hit_state
2nd level key: error_state
2nd level key: violation_state
2nd level key: violation_penalty_state
2nd level key: general_final_state
2nd level key: hit_final_state
2nd level key: error_final_state
2nd level key: violation_final_state
2nd level key: starting_st

In [49]:
# get some info on the peh dict (print in order of nested struture)
itrial = 0
counter = 0
for key, value in peh_dict[itrial].items():
    print(f"1st level key: {key}")
    counter +=1
    for key2, value2 in value.items():
        print(f"2nd level key: {key2}")
        counter+=1
        try: 
            for key3, value3 in value2.items():
                print(f"3rd level key + val: {key3}, {value3}")
                counter+=1
        except:
            None

1st level key: pokes
2nd level key: C
2nd level key: L
2nd level key: R
2nd level key: starting_state
3rd level key + val: C, out
3rd level key + val: L, out
3rd level key + val: R, in
2nd level key: ending_state
3rd level key + val: C, out
3rd level key + val: L, out
3rd level key + val: R, in
1st level key: waves
2nd level key: cp_fixation_wave
2nd level key: sa_sound_wave
2nd level key: sb_sound_wave
2nd level key: go_sound_wave
2nd level key: starting_state
3rd level key + val: cp_fixation_wave, []
3rd level key + val: sa_sound_wave, out
3rd level key + val: sb_sound_wave, out
3rd level key + val: go_sound_wave, out
2nd level key: ending_state
3rd level key + val: cp_fixation_wave, []
3rd level key + val: sa_sound_wave, out
3rd level key + val: sb_sound_wave, out
3rd level key + val: go_sound_wave, out
1st level key: states
2nd level key: state_0
2nd level key: check_next_trial_ready
2nd level key: wait_for_cpoke
2nd level key: cpoke
2nd level key: wait_for_sounds_to_end
2nd level 

Looking at the large print statement above, the `starting` and `ending` state dictionaries output to strings rather than poke, state, or wave in/out time integers. And for `pokes` and `waves` are nested into a third dictionary.

In [51]:
print(f"Number of events on a single trial: {counter}")

Number of events on a single trial: 50


## Example moving from dict values --> long form table


In [59]:
peh_dict[itrial]['pokes']['C'] # one center poke on this trial

array([459.670004, 459.762304])

In [60]:
peh_dict[itrial]['pokes']['starting_state']['C'] # center poke is out at end of trial

'out'

In [64]:
peh_dict[itrial]['states']['wait_for_cpoke'] # two entries into wait_for_cpoke state this trial

array([[448.424204,        nan],
       [448.424304, 459.670004]])

In [65]:
peh_dict[itrial]['states']['ending_state'] # the ending state is state0

'state_0'

Possible long-format [table](https://www.tablesgenerator.com/markdown_tables#):



| animal_id           | sessid                  | date                       | trial                   | event_type                           | event_name                               | entry_num                                     | in_time                    | out_time                 | subevent_name                             | subevent_status                         |
|---------------------|-------------------------|----------------------------|-------------------------|--------------------------------------|------------------------------------------|-----------------------------------------------|----------------------------|--------------------------|-------------------------------------------|-----------------------------------------|
| `str` of  animal id | `int` indicating sessid | `datetime`of  session date | `int` ntrial in session | `str`:  `pokes`, `waves` or `states` | `str`: any 2nd  level key  in `peh_dict` | `int`: size of  time array in 2nd level value | `int`: start time of event | `int`: end time of event | `str`: any 3rd level  key in `peh_dict`** | `str`: value of 3rd level in `peh_dict` |
| R502                | 897749                  | 2022-11-30                 | 1                       | pokes                                | C                                        | 1                                             | 459.6                      | 459.7                    | nan                                       | nan                                     |
| R502                | 897749                  | 2022-11-30                 | 1                       | pokes                                | ending_state                             | 1                                             | nan                        | nan                      | C                                         | out                                     |
| R502                | 897749                  | 2022-11-30                 | 1                       | states                               | wait_for_cpoke                           | 1                                             | 448.4                      | nan                      | nan                                       | nan                                     |
| R502                | 897749                  | 2022-11-30                 | 1                       | states                               | wait_for_cpoke                           | 2                                             | 448.4                      | 459.6                    | nan                                       | nan                                     |
| R502                | 897749                  | 2022-11-30                 | 1                       | states                               | ending_state                             | 1                                             | nan                        | nan                      | state_0                                   | nan                                     |